In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
import numpy as np
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\Quantomatic.jar'
zx.tikz.tikzit_location = r'C:\Users\John\Documents\tikzit\tikzit.exe'

In [4]:
def generate_clifford_circuit(qubits, depth, p_had, p_s):
    p_cnot = 1-p_had-p_s
    c = zx.Circuit(qubits)
    for _ in range(depth):
        r = random.random()
        if r > 1-p_had:
            c.add_gate("HAD",random.randrange(qubits))
        elif r > 1-p_had-p_s:
            c.add_gate("S",random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl!=tgt: break
            c.add_gate("CNOT",tgt,ctrl)
    return c

In [136]:
def normal_form_to_conn(g):
    qs = g.qubits()
    qubits = g.qubit_count()

    gates1 = {}
    verts = {}
    verts_opp = {}

    for i in g.inputs:
        q = qs[i]
        v = list(g.neighbours(i))[0]
        e = g.edge(i,v)
        verts[q] = v
        verts_opp[v] = q
        gates1[q] = ""
        if g.edge_type(e) == 2: gates1[q] += "H"
        phase = g.phase(v)
        if phase == Fraction(1,2): gates1[q] += "S"
        if phase == Fraction(1): gates1[q] += "Z"
        if phase == Fraction(3,2): gates1[q] += "s" # = S*

    for o in g.outputs:
        q = qs[o] + qubits
        v = list(g.neighbours(o))[0]
        e = g.edge(o,v)
        verts[q] = v
        verts_opp[v] = q
        gates1[q] = ""
        if g.edge_type(e) == 2: gates1[q] += "H"
        phase = g.phase(v)
        if phase == Fraction(1,2): gates1[q] += "S"
        if phase == Fraction(1): gates1[q] += "Z"
        if phase == Fraction(3,2): gates1[q] += "s"

    conn = {i:set() for i in range(2*qubits)}
    for i in range(2*qubits):
        v = verts[i]
        for w in g.neighbours(v):
            if w in g.inputs or w in g.outputs: continue
            j = verts_opp[w]
            conn[i].add(j)
            conn[j].add(i)
    return conn, gates1

In [543]:
def combine_single_qubit_gates(s):
    state = ''
    def handle_char(state, c):
        if not state: state = c
        elif c == 'Z':
            if state[-1] == 'Z': state = state[:-1]
            elif state[-1] == 'S': state = state[:-1] + 's'
            elif state[-1] == 's': state = state[:-1] + 'S'
            elif state[-1] in 'VvNH': state += 'Z'
            else: raise Exception(state)
        elif c == 'S':
            if state[-1] == 's': state = state[:-1]
            elif state[-1] == 'S': state = state[:-1] + 'Z'
            elif state[-1] == 'Z': state = state[:-1] + 's'
            elif state[-1] in 'VvNH': state += 'S'
            else: raise Exception(state)
        elif c == 'V':
            if state[-1] == 'S':   state = state[:-1] + 'Hs'
            elif state[-1] == 's': state = state[:-1] + 'HNs'
            elif state[-1] == 'Z': state = state[:-1] + 'vZ'
            elif state[-1] == 'v': state = state[:-1]
            elif state[-1] == 'V': state = state[:-1] +  'N'
            elif state[-1] == 'N': state = state[:-1] + 'v'
            elif state[-1] == 'H': state = state[:-1] + "vs"
            else: raise Exception(state)
        else: raise Exception(state)
        state = state.replace('vH', 'Hs')
        state = state.replace('VH', 'HS')
        state = state.replace('SH', 'HV')
        state = state.replace('sH', 'Hv')
        state = state.replace("NH", "HZ")
        state = state.replace("ZH", "HN")
        state = state.replace("ZN", "NZ")
        state = state.replace("SN", "Ns")
        state = state.replace("sN", "NS")
        state = state.replace("ZV", "vZ")
        state = state.replace("Zv", "VZ")
        state = state.replace("ZS", 's')
        state = state.replace("Zs", "S")
        state = state.replace("SZ", 's')
        state = state.replace("sZ", "S")
        state = state.replace('vv', "N")
        state = state.replace('Nv', 'V')
        state = state.replace('vN', 'V')
        state = state.replace('NV', 'v')
        state = state.replace('VN', 'v')
        kill = ['vV','Vv','sS','Ss','NN','ZZ','HH']
        for k in kill: state = state.replace(k,'')
        return state
    
    for c in s:
        if c == 's':
            state = handle_char(state, 'Z')
            state = handle_char(state,'S')
        elif c == 'H':
            state = handle_char(state,'S')
            state = handle_char(state,'V')
            state = handle_char(state,'S')
        else:
            state = handle_char(state,c)
    state = state.replace("Hvs", "V")
    state = state.replace("HVS", "v")
    state = state.replace("Hsv", "S")
    state = state.replace("HSV", "s")
    return state

def conn_to_matrix(conn):
    qubits = len(gates1)//2
    data = []
    for q in range(qubits):
        row = [0]*qubits
        for i in conn[q]:
            if i>=qubits: row[i-qubits] = 1
        data.append(row)
    return zx.Mat2(data)

def circ_from_conn(conn, gates1):
    qubits = len(gates1)//2
    inputs = list(range(qubits))
    outputs = list(range(qubits,2*qubits))
    c = zx.Circuit(qubits)
    for q in range(qubits):
        for s in combine_single_qubit_gates(gates1[q]):
            if s == "H": gate = "HAD"
            if s == "S": gate = "S"
            if s == "Z": gate = "Z"
            if s == "N": gate = "NOT"
            if s == "s": c.add_gate("S",q,adjoint=True)
            elif s == "V": c.add_gate("XPhase",q, Fraction(1,2))
            elif s == "v": c.add_gate("XPhase",q, Fraction(3,2))
            else: c.add_gate(gate, q)
    
    for q in range(qubits):
        for i in conn[q]:
            if i<q: c.add_gate("CZ",i,q)
    m = conn_to_matrix(conn)
    #print(m)
    #m.transpose()
    cnots = m.to_cnots(optimize=True)
    #print(cnots)
    for cnot in cnots: 
        c.add_gate(cnot)
        #c.add_gate("CNOT", cnot.control, cnot.target)
    for q in range(qubits): c.add_gate("HAD", q)
    for q in range(qubits):
        for i in conn[q+qubits]:
            if i>q+qubits: c.add_gate("CZ",q,i-qubits)
    for q in range(qubits):
        for s in reversed(combine_single_qubit_gates(gates1[q+qubits])):
            if s == "H": gate = "HAD"
            if s == "S": gate = "S"
            if s == "Z": gate = "Z"
            if s == "s": c.add_gate("S",q,adjoint=True)
            elif s == "V": c.add_gate("XPhase",q, Fraction(1,2))
            elif s == "v": c.add_gate("XPhase",q, Fraction(3,2))
            else: c.add_gate(gate, q)
    return c

In [515]:
def local_comp(i, conn, gates1, dummy=False):
    # Do local complementation on vertex labelled by i, connectivity specified by conn, and single qubit gates gates1
    if not dummy: gates1[i] += "V"
    for j1 in conn[i]:
        for j2 in conn[i]:
            if j1 >= j2: continue # prevent duplicates
            if j2 in conn[j1]:
                conn[j1].remove(j2)
                conn[j2].remove(j1)
            else:
                conn[j1].add(j2)
                conn[j2].add(j1)
        if not dummy: gates1[j1] += "s"

In [636]:
cz_weight = 5
cnot_weight = 1

def get_score(conn):
    qubits = len(conn)//2
    inputs = list(range(qubits))
    outputs = list(range(qubits,2*qubits))
    czs = 0
    for i in conn:
        for j in conn[i]:
            if j <= i: continue
            if (i in inputs and j in inputs) or (i in outputs and j in outputs):
                #val += cz_weight
                czs += 1
            #else: val += cnot_weight
    m = conn_to_matrix(conn)
    return czs + len(m.to_cnots(optimize=True))

def find_best_local_comp(conn):
    qubits = len(conn)//2
    inputs = list(range(qubits))
    outputs = list(range(qubits,2*qubits))
    scores = {}
    base = get_score(conn)
    best_score = base
    best = -1
    for i in conn:
        local_comp(i, conn, None, dummy=True)
        score = get_score(conn)
        scores[i] = score
        if score < best_score:
            best_score = score
            best = i
        local_comp(i, conn, None, dummy=True)
    
    return best, best_score, base

def repeat_local_comps(conn, gates1, quiet=True):
    best, s1, s2 = find_best_local_comp(conn)
    while True:
        if best == -1: break
        if not quiet: print(best, s2)
        local_comp(best, conn, gates1)
        best, s1, s2 = find_best_local_comp(conn)

def anneal(conn, gates1, T, beta, quiet=True):
    base = get_score(conn)
    temp = T*base/100
    #beta = temp*beta
    
    def accept(i):
        local_comp(i, conn, None, dummy=True)
        score = get_score(conn)
        local_comp(i, conn, None, dummy=True)
        if score < base: return True
        return (math.exp(-(score-base)/temp) > random.random())
    
    steps_taken = 0
    iterations = 0
    
    while temp > 1:
        iterations += 1
        poss = list(range(len(conn)))
        random.shuffle(poss)
        i = poss.pop()
        if accept(i):
            local_comp(i, conn, gates1)
            new = get_score(conn)
            if not quiet: print(base, new, temp)
            base = new
            steps_taken += 1
            poss = list(range(len(conn)))
            random.shuffle(poss)
        if iterations % len(conn)//4 == 0:
            temp = temp*beta
    if not quiet: print("steps taken:", steps_taken)
    #repeat_local_comps(conn,gates1,quiet=quiet)

In [637]:
seed = 1339
random.seed(seed)
reps = 1
avg = 0
avg2 = 0
for i in range(1,reps+1):
    if i%5 == 0: print(i,end='. ')
    c = generate_clifford_circuit(15,350,0.3,0.2)
    cp = zx.optimize.basic_optimization(c.split_phase_gates(),do_swaps=False)
    g = c.to_graph()
    zx.simplify.clifford_simp(g,quiet=True)
    g.normalise()
    conn, gates1 = normal_form_to_conn(g)
    c2 = circ_from_conn(conn,gates1)
    anneal(conn,gates1,15,0.99,quiet=False)
    c3 = circ_from_conn(conn,gates1)
    improvement = (c2.twoqubitcount() - c3.twoqubitcount())/c2.twoqubitcount()
    avg += improvement
    c4 = zx.optimize.basic_optimization(c3.split_phase_gates(),do_swaps=False)
    improvement = (cp.twoqubitcount() - c4.twoqubitcount())/cp.twoqubitcount()
    avg2 += improvement
print(avg/reps)
print(avg2/reps)

214 214 32.1
214 200 31.779
200 206 31.46121
206 196 31.1465979
196 198 31.1465979
198 192 31.1465979
192 207 31.1465979
207 202 31.1465979
202 191 31.1465979
191 193 31.1465979
193 184 31.1465979
184 193 31.1465979
193 195 31.1465979
195 197 31.1465979
197 217 31.1465979
217 212 31.1465979
212 200 31.1465979
200 193 31.1465979
193 187 31.1465979
187 184 31.1465979
184 197 31.1465979
197 183 31.1465979
183 183 31.1465979
183 188 31.1465979
188 194 31.1465979
194 211 31.1465979
211 207 31.1465979
207 192 31.1465979
192 216 31.1465979
216 213 30.835131921
213 205 30.526780601789998
205 204 30.221512795772096
204 186 29.919297667814373
186 181 29.919297667814373
181 194 29.919297667814373
194 198 29.919297667814373
198 198 29.919297667814373
198 196 29.919297667814373
196 196 29.919297667814373
196 184 29.919297667814373
184 189 29.919297667814373
189 180 29.919297667814373
180 193 29.919297667814373
193 200 29.919297667814373
200 197 29.919297667814373
197 203 29.919297667814373
203 193 

183 183 19.226487807563768
183 168 19.226487807563768
168 176 19.226487807563768
176 187 19.226487807563768
187 199 19.226487807563768
199 191 19.226487807563768
191 203 19.226487807563768
203 191 18.84388070019325
191 186 18.468887474259404
186 183 18.468887474259404
183 189 18.468887474259404
189 200 18.468887474259404
200 187 18.468887474259404
187 200 18.468887474259404
200 197 18.468887474259404
197 189 18.468887474259404
189 181 18.468887474259404
181 181 18.468887474259404
181 178 18.468887474259404
178 192 18.468887474259404
192 197 18.468887474259404
197 184 18.468887474259404
184 175 18.468887474259404
175 185 18.468887474259404
185 180 18.468887474259404
180 186 18.468887474259404
186 194 18.468887474259404
194 200 18.468887474259404
200 196 18.468887474259404
196 194 18.468887474259404
194 195 18.468887474259404
195 198 18.468887474259404
198 202 18.28419859951681
202 203 18.10135661352164
203 189 17.920343047386424
189 199 17.74113961691256
199 198 17.74113961691256
198 19

192 190 10.95143057460076
190 192 10.95143057460076
192 191 10.95143057460076
191 192 10.95143057460076
192 191 10.95143057460076
191 192 10.95143057460076
192 202 10.95143057460076
202 204 10.95143057460076
204 181 10.95143057460076
181 182 10.841916268854753
182 180 10.733497106166205
180 170 10.626162135104542
170 176 10.519900513753496
176 185 10.519900513753496
185 178 10.519900513753496
178 190 10.519900513753496
190 178 10.519900513753496
178 183 10.519900513753496
183 188 10.519900513753496
188 192 10.519900513753496
192 191 10.519900513753496
191 185 10.519900513753496
185 189 10.519900513753496
189 191 10.519900513753496
191 191 10.519900513753496
191 193 10.519900513753496
193 191 10.519900513753496
191 196 10.519900513753496
196 198 10.519900513753496
198 200 10.519900513753496
200 193 10.414701508615961
193 193 10.207448948594504
193 188 10.105374459108559
188 183 10.105374459108559
183 192 10.105374459108559
192 198 10.105374459108559
198 209 10.105374459108559
209 206 10

192 191 5.529222759147189
191 201 5.529222759147189
201 191 5.529222759147189
191 181 5.473930531555718
181 176 5.3649993139777585
176 174 5.311349320837981
174 177 5.311349320837981
177 175 5.311349320837981
175 175 5.311349320837981
175 186 5.311349320837981
186 187 5.311349320837981
187 195 5.311349320837981
195 193 5.311349320837981
193 182 5.311349320837981
182 183 5.311349320837981
183 182 5.311349320837981
182 195 5.311349320837981
195 194 5.311349320837981
194 192 5.311349320837981
192 193 5.311349320837981
193 188 5.153596934659772
188 186 5.102060965313174
186 185 5.102060965313174
185 179 5.102060965313174
179 181 5.102060965313174
181 186 5.102060965313174
186 189 5.102060965313174
189 185 5.102060965313174
185 182 5.102060965313174
182 176 5.102060965313174
176 181 5.102060965313174
181 179 5.000529952103442
179 184 4.901019406056583
184 183 4.901019406056583
183 186 4.901019406056583
186 186 4.901019406056583
186 179 4.901019406056583
179 188 4.901019406056583
188 185 4.9

In [638]:
print(c.stats())
cp = zx.optimize.basic_optimization(c.split_phase_gates(),do_swaps=False)
print(cp.stats())

Circuit  on 15 qubits with 350 gates.
        0 is the T-count
        350 Cliffords among which 
        184 2-qubit gates and 94 Hadamard gates.
Circuit  on 15 qubits with 292 gates.
        0 is the T-count
        292 Cliffords among which 
        173 2-qubit gates and 48 Hadamard gates.


In [640]:
zx.d3.draw(cp)

In [639]:
print(c3.stats())
c4 = zx.optimize.basic_optimization(c3.split_phase_gates(),do_swaps=False)
print(c4.stats())

Circuit  on 15 qubits with 235 gates.
        0 is the T-count
        235 Cliffords among which 
        161 2-qubit gates and 36 Hadamard gates.
Circuit  on 15 qubits with 234 gates.
        0 is the T-count
        234 Cliffords among which 
        153 2-qubit gates and 34 Hadamard gates.


In [641]:
zx.d3.draw(c4)

In [628]:
print(c2.stats())
c2p = zx.optimize.basic_optimization(c2.split_phase_gates(),do_swaps=False)
print(c2p.stats())

Circuit  on 15 qubits with 256 gates.
        0 is the T-count
        256 Cliffords among which 
        202 2-qubit gates and 31 Hadamard gates.
Circuit  on 15 qubits with 262 gates.
        0 is the T-count
        262 Cliffords among which 
        188 2-qubit gates and 26 Hadamard gates.


In [156]:
zx.compare_tensors(c2,c3,preserve_scalar=False)

True

In [133]:
zx.d3.draw(c3)

In [134]:
zx.d3.draw(c2)